This notebook set's up an FEM simulation by sampling from the analytical approximation of the surface patch with some $\theta$. Then we translate this into a simnibs input matrix and run a simulation and return the results

In [2]:
import sys

In [3]:
#SCC package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/Python_modules/src')

In [148]:
import numpy as np
from simnibs import sim_struct, run_simulation

#For importing geometry library
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

#Timing
from timeit import default_timer as timer

In [5]:
#https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        
        
        path = os.path.join(self.path,fullname) + '.ipynb'

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [135]:
#Load up the geolibrary as a module
glib_path = '/projects/jjeyachandra/rtms_optimize/notebooks/python2/'
nbload = NotebookLoader(glib_path)
geolib = nbload.load_module('geolib')

importing Jupyter notebook from /projects/jjeyachandra/rtms_optimize/notebooks/python2/geolib.ipynb


In [136]:
#Load up quadratic surface constants, inverse rotation matrix, bounds and normal vector
C = np.fromfile('../../output/quadratic_vec')
iR = np.fromfile('../../output/inverse_rot').reshape((3,3))
b  = np.fromfile('../../output/param_bounds').reshape((3,2))
n = np.fromfile('../../output/norm_varr')
n = n/np.linalg.norm(n)

### Get raw input coordinates that need to be transformed

In [137]:
#Sample a coordinate within the bounds and form input array
x = np.random.uniform(b[0,0], b[0,1])
y = np.random.uniform(b[1,0], b[1,1])

#Sample rotation
r = np.random.uniform(0,180)
print(x,y,r)

(27.708634082292754, 8.16081378804095, 19.020825067538652)


In [138]:
#Compute z value then map to mesh surface
preaff_loc = geolib.map_param_2_surf(x,y,C)
loc = np.matmul(iR,preaff_loc)

#Compute rotation vector then align with mesh surface
preaff_rot = geolib.map_rot_2_surf(x,y,r,C)
rot = np.matmul(iR,preaff_rot)

In [139]:
#Start a session
S = sim_struct.SESSION()
S.fnamehead="../../data/simnibs_output/sub-CMH090.msh"
S.pathfem="../../data/simnibs_output/test_simulation"

In [140]:
#Define the TMS simulation
tms = S.add_tmslist()
tms.fnamecoil='/projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz'

In [141]:
#Define position
pos = tms.add_position()
pos.matsimnibs = geolib.define_coil_orientation(loc,rot,n)

In [151]:
pos.matsimnibs

array([[ -5.2500e-02,  -6.2950e-01,   7.7522e-01,  -4.7358e+01],
       [  5.3868e-01,  -6.7152e-01,  -5.0881e-01,   5.7430e+01],
       [ -8.4087e-01,  -3.9088e-01,  -3.7436e-01,   4.8858e+01],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   1.0000e+00]])

In [150]:
start = timer()
run_simulation(S)
stop = timer()
print(stop - start)

[ simnibs ]INFO: Could not find the dti tensor file in the standard location: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/d2c_sub-CMH090/dti_results_T1space/DTI_conf_tensor.nii.gz
[ simnibs ]INFO: Head Mesh: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/sub-CMH090.msh
[ simnibs ]INFO: Subject Path: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/m2m_sub-CMH090
[ simnibs ]INFO: Simulation Folder: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation
[ simnibs ]INFO: Running simulations in the directory: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Using coil: /projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities


0


[ simnibs ]INFO: Running TMS Position 0
[ simnibs ]INFO: Coil Position Matrix: [[ -5.2500e-02  -6.2950e-01   7.7522e-01  -4.7358e+01]
 [  5.3868e-01  -6.7152e-01  -5.0881e-01   5.7430e+01]
 [ -8.4087e-01  -3.9088e-01  -3.7436e-01   4.8858e+01]
 [  0.0000e+00   0.0000e+00   0.0000e+00   1.0000e+00]]
dIdt: 1000000.0

[ simnibs ]INFO: Executing: 
/home/jjeyachandra/simnibs_2.1.2/bin/getdp /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation/tmp5LaDSv.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin
[ simnibs ]INFO: Info    : Running '/home/jjeyachandra/simnibs_2.1.2/bin/getdp /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_outp

[ simnibs ]INFO: Info    :  45 KSP Residual norm 1.297759710242e-03
[ simnibs ]INFO: Info    :  46 KSP Residual norm 1.081072616306e-03
[ simnibs ]INFO: Info    :  47 KSP Residual norm 8.358478506424e-04
[ simnibs ]INFO: Info    :  48 KSP Residual norm 6.677135118278e-04
[ simnibs ]INFO: Info    :  49 KSP Residual norm 5.178251526009e-04
[ simnibs ]INFO: Info    :  50 KSP Residual norm 3.977936833804e-04
[ simnibs ]INFO: Info    :  51 KSP Residual norm 3.150377467341e-04
[ simnibs ]INFO: Info    :  52 KSP Residual norm 2.517821261615e-04
[ simnibs ]INFO: Info    :  53 KSP Residual norm 2.048426288662e-04
[ simnibs ]INFO: Info    :  54 KSP Residual norm 1.637676946081e-04
[ simnibs ]INFO: Info    :  55 KSP Residual norm 1.357617382566e-04
[ simnibs ]INFO: Info    :  56 KSP Residual norm 1.210466748713e-04
[ simnibs ]INFO: Info    :  57 KSP Residual norm 1.019513026368e-04
[ simnibs ]INFO: Info    :  58 KSP Residual norm 9.035369199148e-05
[ simnibs ]INFO: Info    :  59 KSP Residual norm

271.975129128
